# Conditional Probability experiment

* we take all numeric features and split them into buckets
* use Theil's U as categorical feature selector
* measure the conditional probability of our target class when being in a certain bucket (for each feature)
* we average these probabilities out
* we take the softmax to scale the output
* we do this in multiple iterations and take a final average.


# Load the data

In [1]:
import numpy as np
import pandas as pd
import re
from typing import Optional, Tuple, Union

import matplotlib.pyplot as plt
from xgboost import plot_tree

import warnings
warnings.filterwarnings("ignore")


from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.preprocessing import PowerTransformer, LabelEncoder

In [2]:
train = pd.read_csv('/kaggle/input/credit-card-fraud-prediction/train.csv')
test = pd.read_csv('/kaggle/input/credit-card-fraud-prediction/test.csv')
submission = pd.read_csv('/kaggle/input/credit-card-fraud-prediction/sample_submission.csv')

#cols = df_orig.columns
#train = pd.concat((train, df_orig), axis=0).reset_index(drop=True)

print('The dimension of the train dataset is:', train.shape)
print('The dimension of the test dataset is:', test.shape)

The dimension of the train dataset is: (150000, 32)
The dimension of the test dataset is: (69129, 31)


In [3]:
train

,id,Time,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,...,feat21,feat22,feat23,feat24,feat25,feat26,feat27,feat28,Transaction_Amount,IsFraud
0,0,0.0,2.074329,-0.129425,-1.137418,0.412846,-0.192638,-1.210144,0.110697,-0.263477,...,-0.334701,-0.887840,0.336701,-0.110835,-0.291459,0.207733,-0.076576,-0.059577,1.98,0
1,1,0.0,1.998827,-1.250891,-0.520969,-0.894539,-1.122528,-0.270866,-1.029289,0.050198,...,0.054848,-0.038367,0.133518,-0.461928,-0.465491,-0.464655,-0.009413,-0.038238,84.00,0
2,2,0.0,0.091535,1.004517,-0.223445,-0.435249,0.667548,-0.988351,0.948146,-0.084789,...,-0.326725,-0.803736,0.154495,0.951233,-0.506919,0.085046,0.224458,0.087356,2.69,0
3,3,0.0,1.979649,-0.184949,-1.064206,0.120125,-0.215238,-0.648829,-0.087826,-0.035367,...,-0.095514,-0.079792,0.167701,-0.042939,0.000799,-0.096148,-0.057780,-0.073839,1.00,0
4,4,0.0,1.025898,-0.171827,1.203717,1.243900,-0.636572,1.099074,-0.938651,0.569239,...,0.099157,0.608908,0.027901,-0.262813,0.257834,-0.252829,0.108338,0.021051,1.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,149995,73899.0,1.277125,0.665683,-0.688148,1.135626,0.494826,-0.554938,0.252478,-0.132739,...,-0.114747,-0.221548,-0.233038,-0.744995,0.799359,-0.244856,0.037868,0.042670,1.00,0
149996,149996,73899.0,0.807735,-1.813163,0.421073,-0.576839,-1.601656,-0.665517,-0.230382,-0.297095,...,0.270879,0.014436,-0.193669,0.387714,0.169280,-0.444501,-0.043788,0.072515,349.70,0
149997,149997,73899.0,0.836403,-0.351598,0.650338,1.066155,-0.234826,0.844271,-0.409578,0.382619,...,0.285914,0.721022,-0.067894,-0.273675,0.232969,-0.286735,0.054066,0.031396,116.10,0
149998,149998,73899.0,-0.806965,0.383847,2.296469,1.428714,-2.343948,1.073324,-0.203567,0.456589,...,0.228740,1.296798,-0.038753,0.827484,-0.743000,0.914488,0.307576,-0.010200,179.00,0


In [4]:
test

,id,Time,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,...,feat20,feat21,feat22,feat23,feat24,feat25,feat26,feat27,feat28,Transaction_Amount
0,150000,73899,-0.681234,-0.371212,0.385966,0.955703,2.064078,0.338827,-0.539452,-0.254046,...,0.183560,-0.253324,0.266668,-0.153829,0.455969,-0.503628,0.257588,-0.456685,-0.298919,30.42
1,150001,73899,-1.098947,-0.959377,0.324934,0.703908,1.090582,-1.595909,0.584548,0.260069,...,0.334764,0.130108,0.676928,-0.161070,-0.638011,-0.273424,0.711132,0.349967,0.141233,23.00
2,150002,73899,0.977029,-0.270984,0.471526,-1.232570,0.957537,-0.636602,-0.953060,-1.491744,...,0.355728,0.517912,1.175087,-0.325895,-0.362636,0.306037,0.004828,0.037389,0.058222,198.00
3,150003,73900,1.176658,-0.225816,-0.246600,0.015513,1.103831,1.229516,-1.527098,-0.459769,...,-0.152613,-0.104600,0.003800,-0.023180,-0.458338,0.481427,-0.381415,0.080165,0.027372,9.99
4,150004,73900,0.804828,-0.272967,0.959910,-1.117567,0.395748,0.589855,1.059880,-1.101203,...,-0.127818,-0.011849,0.078540,-0.241700,0.010130,0.302614,-0.259568,0.023127,0.056957,239.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69124,219124,120580,1.891079,-0.041423,0.858158,-1.272908,-3.783908,-1.388939,-0.280639,2.012789,...,-0.195703,-0.181369,-0.456538,-0.069571,0.756765,0.244479,-0.147566,-0.054725,-0.044588,198.65
69125,219125,120580,0.139724,-0.917395,-0.257933,0.948649,-2.913655,-2.184829,-0.457534,1.883716,...,-0.116538,0.491469,1.478823,-0.085398,-0.091409,-1.053488,0.467570,0.358918,0.294735,24.00
69126,219126,120580,2.058343,0.391801,-0.136498,-0.038993,-1.928553,0.330117,0.179926,0.270127,...,-0.384830,-0.306640,-0.965783,0.307799,-0.021434,-0.343989,0.181065,-0.098387,-0.044064,1.79
69127,219127,120580,2.079227,0.301966,1.536193,-2.162389,-1.785833,-2.804889,-0.058879,0.552845,...,-0.190984,0.109909,0.590401,0.286621,0.675660,-0.510736,-0.090044,0.056749,-0.017126,88.00


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 32 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  150000 non-null  int64  
 1   Time                150000 non-null  float64
 2   feat1               150000 non-null  float64
 3   feat2               150000 non-null  float64
 4   feat3               150000 non-null  float64
 5   feat4               150000 non-null  float64
 6   feat5               150000 non-null  float64
 7   feat6               150000 non-null  float64
 8   feat7               150000 non-null  float64
 9   feat8               150000 non-null  float64
 10  feat9               150000 non-null  float64
 11  feat10              150000 non-null  float64
 12  feat11              150000 non-null  float64
 13  feat12              150000 non-null  float64
 14  feat13              150000 non-null  float64
 15  feat14              150000 non-nul

In [6]:
train.describe()

,id,Time,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,...,feat21,feat22,feat23,feat24,feat25,feat26,feat27,feat28,Transaction_Amount,IsFraud
count,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,...,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000
mean,74999.500000,49785.422340,0.069456,-0.003674,0.739860,0.104810,-0.254623,0.151934,-0.165443,0.166254,...,-0.032077,-0.068876,-0.059646,-0.000776,0.161152,0.021575,0.014138,0.020323,71.065194,0.001793
std,43301.414527,18733.655609,1.346553,1.161502,1.012801,1.230909,1.048268,1.182964,0.802668,0.691267,...,0.404916,0.573755,0.323272,0.584270,0.385175,0.473795,0.223692,0.163429,158.712510,0.042310
min,0.000000,0.000000,-22.173926,-44.247914,-19.088980,-5.069484,-21.090735,-25.659750,-31.179799,-28.903442,...,-14.689621,-7.351378,-11.958588,-2.827444,-3.958591,-1.413857,-9.234767,-4.454059,0.000000,0.000000
25%,37499.750000,39411.000000,-0.844248,-0.621724,0.181373,-0.738399,-0.912254,-0.590892,-0.667139,-0.078114,...,-0.185077,-0.464426,-0.177546,-0.321495,-0.078682,-0.310948,-0.044800,0.001623,6.990000,0.000000
50%,74999.500000,53785.000000,0.671327,0.003677,0.826497,0.095342,-0.322525,-0.068690,-0.144551,0.125052,...,-0.044828,-0.045844,-0.068170,0.043048,0.181701,-0.087927,0.017590,0.024098,24.900000,0.000000
75%,112499.250000,63833.000000,1.171271,0.753340,1.360362,0.967657,0.242796,0.492931,0.334553,0.396056,...,0.100896,0.309938,0.047399,0.388969,0.425791,0.269844,0.073776,0.065778,74.000000,0.000000
max,149999.000000,73899.000000,2.390816,16.068473,6.145578,12.547997,34.581260,15.082516,23.256066,18.270586,...,22.062945,6.163541,12.734391,4.572739,3.111624,3.119824,6.457839,23.263746,6513.350000,1.000000


# Feature engineering

In this section we will build features using various techniques.

In [7]:
target = "IsFraud"

In [8]:
train[target].value_counts()

IsFraud
0    149731
1       269
Name: count, dtype: int64

In [9]:
# add binary features based on shap dependency plots
def add_binary_flags(df: pd.DataFrame) -> pd.DataFrame:
    df["feat9_feat6_binary"] = ((df["feat9"] > 0) & (df["feat6"] > 1))
    df["feat4_feat13_binary"] = ((df["feat4"] > 0) & (df["feat13"] > 0.5))
    df["Transaction_Amount_feat9_binary"] = ((df["Transaction_Amount"] > 0) & (df["feat13"] > 1.2))
    df["feat18_feat19_binary"] = ((df["feat18"] > 0) & (df["feat19"] <= 0.5) & (df["feat19"] >= -0.5))
    df["feat18_feat20_binary"] = ((df["feat18"] > 0) & (df["feat20"] <= 0.5) & (df["feat20"] >= -0.5))
    return df

In [10]:
train = add_binary_flags(train)
test = add_binary_flags(test)

In [11]:
train = train.drop("id", axis=1)
test = test.drop("id", axis=1)

# ProbaBoost

In [12]:
import os
os.environ["OMP_NUM_THREADS"] = "1" # export OMP_NUM_THREADS=1
os.environ["OPENBLAS_NUM_THREADS"] = "1" # export OPENBLAS_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] = "1" # export MKL_NUM_THREADS=1
os.environ["VECLIB_MAXIMUM_THREADS"] = "1" # export VECLIB_MAXIMUM_THREADS=1
os.environ["NUMEXPR_NUM_THREADS"] = "1" # export NUMEXPR_NUM_THREADS=1

from sklearn import preprocessing
import numpy as np
import gc
from collections import Counter
import pandas as pd
import scipy.stats as ss
from numpy import dot
from numpy.linalg import norm
import math
from sklearn.metrics import matthews_corrcoef
from joblib import Parallel, delayed
import optuna



from sklearn import preprocessing
import numpy as np
import gc
from collections import Counter
import pandas as pd
import scipy.stats as ss
from numpy import dot
from numpy.linalg import norm
import math


class ConditionProbabilitiesClassifier:
    """
    This classifier measures the conditional probability of the target variable for each feature
    separately. Numerical values are converted into buckets for this purpose.
    Additionally, the explanatory power of each feature is measured using Theil's U. The conditional probabilities
    are then weighted by Theil's U.
    :param df: Pandas DataFrame containing the target feature.
    :param cat_feats: List of strings with column names specifying categorical features.
    :param num_feats: List of strings with column names specifying numerical features.
    :param target: String specifying name of the target column.
    """

    def __init__(self,
                 df: pd.DataFrame,
                 cat_feats: list,
                 num_feats: list,
                 target: str,
                 nb_buckets: int = 5):
        self.df = df.reset_index(drop=True)
        self.cat_feats = cat_feats
        self.num_feats = num_feats
        self.target = target
        self.nb_buckets = nb_buckets
        
        self.thresholds = np.linspace(0, 1, self.nb_buckets).tolist()
        self.thresholds.append(2000000)
        self.num_buckets = [f"< {threshold}" for threshold in self.thresholds]
        self.cat_cond_probs = {}
        self.num_cond_probs = {}
        self.cat_theil_u_weights = {}
        self.num_theil_u_weights = {}
        self.cat_selected_features = []
        self.num_selected_features = []
        self.scalers = {}

    def fill_nulls(self, df: pd.DataFrame, col: str, fill_with=None, coltype: str = 'str'):
        """
        Fill nulls with provided term.
        :param df: Pandas DaaFrame
        :param col: String specifying column name.
        :param fill_with: String, integer or float to fill missing values with.
        :param coltype: One of 'str' or 'float'. Will be used to typecast the column.
        """
        if coltype == 'str':
            df[col] = df[col].astype(str)
        elif coltype == 'float':
            df[col] = df[col].astype(np.float64)

        df[col] = df[col].fillna(fill_with)
        return df

    def conditional_entropy(self, x, y):
        # entropy of x given y
        y_counter = Counter(y)
        xy_counter = Counter(list(zip(x, y)))
        total_occurrences = sum(y_counter.values())
        entropy = 0
        for xy in xy_counter.keys():
            p_xy = xy_counter[xy] / total_occurrences
            p_y = y_counter[xy[1]] / total_occurrences
            entropy += p_xy * math.log(p_y / p_xy)
        return entropy

    def theil_u(self, x, y):
        s_xy = self.conditional_entropy(x, y)
        x_counter = Counter(x)
        total_occurrences = sum(x_counter.values())
        p_x = list(map(lambda n: n / total_occurrences, x_counter.values()))
        s_x = ss.entropy(p_x)
        if s_x == 0:
            return 1
        else:
            return (s_x - s_xy) / s_x

    def fit_theil_u_weights(self):
        """
        Loops through categorical and numerical columns and measured their target explainability
        using Theil's U.
        :return: Updates class attributes.
        """
        for col in self.cat_feats:
            theil_u = self.theil_u(self.df[col], self.df[self.target])
            self.cat_theil_u_weights[col] = theil_u

        for col in self.num_feats:
            theil_u = self.theil_u(self.df[col], self.df[self.target])
            self.num_theil_u_weights[col] = theil_u

    def fit_theil_u_feature_selection(self):
        """
        Loops through all columns and selects columns with Theil U value of 0.01 or higher.
        :return: Updates selected_features attribute.
        """
        for col in self.cat_feats:
            if self.cat_theil_u_weights[col] >= 0.00:
                self.cat_selected_features.append(col)

        for col in self.num_feats:
            if self.num_theil_u_weights[col] >= 0.00:
                self.num_selected_features.append(col)

    def fit_categorical_cond_probas(self):
        """
        Loops through all categorical features and their categories
        and maps the conditional probability of the target.
        :return: Updates class attributes.
        """
        for col in self.cat_feats:
            self.cat_cond_probs[col] = {}
            for category in self.df[col].unique():
                temp_df = self.df.loc[self.df[col] == category]
                cond_prob = (temp_df[self.target].sum() / len(self.df.index)) / (  # joint prob
                        len(temp_df.index) / len(self.df.index))  # marginal
                self.cat_cond_probs[col][category] = cond_prob

            if "Unknown" not in self.cat_cond_probs[col].keys():
                self.cat_cond_probs[col]["Unknown"] = 0

    def fit_predict_buckets(self, col: str):
        """
        Takes a column of a numerical type and returns it as a bucketed version.
        :param col: String specifying which column to scale and return as bucketed version.
        :return: Pandas DataFrame
        """
        # scale Series
        scaler = preprocessing.MinMaxScaler()
        scaled = scaler.fit_transform(self.df[[col]])
        scaled = pd.DataFrame(scaled, columns=[col])
        self.scalers[col] = scaler
        
        # add random noise to also occupy highest bucket
        scaled += 0.001

        # transform into buckets
        conditions = []
        for thres in self.thresholds:
            conditions.append((scaled[col] < thres))
        
        choices = self.num_buckets

        scaled[col] = np.select(conditions, choices, default='Unknown')
        return scaled

    def predict_buckets(self, df, col: str):
        """
        Takes a Pandas Series of a numerical type and returns it as a bucketed version.
        :param col: String specifying which column to scale and return as bucketed version.
        returns: Pandas DataFrame
        """
        # scale Series
        scaler = self.scalers[col]
        scaled = scaler.transform(df[[col]])
        scaled = pd.DataFrame(scaled, columns=[col])

        # transform into buckets
        conditions = []
        for thres in self.thresholds:
            conditions.append((scaled[col] < thres))

        choices = self.num_buckets

        scaled[col] = np.select(conditions, choices, default='Unknown')
        return scaled

    def fit_numerical_cond_probas(self):
        """
        Loops through all numerical features and their buckets
        and maps the conditional probabilities of the target.
        """
        for col in self.num_feats:
            bucketed = self.fit_predict_buckets(col=col)
            bucketed[self.target] = self.df[self.target]
            self.num_cond_probs[col] = {}
            cond_prob = 0
            for bucket in self.num_buckets:
                temp_df = bucketed.loc[bucketed[col] == bucket]
                if temp_df.empty:
                    pass
                else:
                    cond_prob = (temp_df[self.target].sum() / len(self.df.index)) / (  # joint prob
                            len(temp_df.index) / len(self.df.index))  # marginal
                self.num_cond_probs[col][bucket] = cond_prob

            if "Unknown" not in self.num_cond_probs[col].keys():
                self.num_cond_probs[col]["Unknown"] = 0

            del bucketed
            _ = gc.collect()
            
    def softmax(self, x):
        """Compute softmax values for each sets of scores in x."""
        return np.exp(x) / np.sum(x ** 2.1, axis=0)  # softmax is adjusted here to improve the balanced logloss

    def fit(self):
        """
        Wrapper function to fit conditional probabilities
        of categorial and numerical features. Updates the
        dictionaries self.cat_cond_probs and self.num_cond_probs.
        """
        for col in self.cat_feats:
            self.df = self.fill_nulls(self.df, col, fill_with='Unknown', coltype='str')

        for col in self.num_feats:
            self.df = self.fill_nulls(self.df, col, fill_with=0, coltype='float')
            self.df = self.df.replace([np.inf, -np.inf], 0)

        self.fit_categorical_cond_probas()
        self.fit_numerical_cond_probas()
        self.fit_theil_u_weights()
        self.fit_theil_u_feature_selection()

    def fit_predict(self):
        """
        Wrapper function to fit conditional probabilities
        of categorial and numerical features. Updates the
        dictionaries self.cat_cond_probs and self.num_cond_probs.
        """
        self.fit()

        # transform cols into conditional probas
        for col in self.cat_selected_features:
            for category in self.df[col].unique():
                if category not in self.cat_cond_probs[col].keys():
                    self.cat_cond_probs[col][category] = 0
            self.df = self.df.replace(self.cat_cond_probs[col])
            #self.df[col] = self.df[col] * self.cat_theil_u_weights[col]

        for col in self.num_selected_features:
            self.df[col] = self.predict_buckets(self.df, col)
            self.df = self.df.replace(self.num_cond_probs[col])
            #self.df[col] = self.df[col] * self.num_theil_u_weights[col]

        # get predictions
        preds = self.df[self.cat_selected_features + self.num_selected_features].mean(axis=1)
        self.df["preds_class_1"] = preds
        self.df["preds_class_0"] = 1 - preds
        preds = pd.DataFrame(self.softmax(self.df[["preds_class_1", "preds_class_0"]].values), columns=["preds_class_1", "preds_class_0"])
        return preds["preds_class_1"]

    def predict(self, df: pd.DataFrame):
        """
        Create predictions based on conditional probabilities and their Theil's U weights.
        :param df: Pandas DataFrame for prediction.
        :return Pandas Series
        """
        # transform cols into conditional probas
        for col in self.cat_selected_features:
            df = self.fill_nulls(df, col, fill_with='Unknown')
            for category in df[col].unique():
                if category not in self.cat_cond_probs[col].keys():
                    self.cat_cond_probs[col][category] = 0
            df = df.replace(self.cat_cond_probs[col])
            #df[col] = df[col] * self.cat_theil_u_weights[col]

        for col in self.num_selected_features:
            df = self.fill_nulls(df, col, fill_with=0, coltype='float')
            df = df.replace([np.inf, -np.inf], 0)
            df[col] = self.predict_buckets(df, col)
            df = df.replace(self.num_cond_probs[col])
            #df[col] = df[col] * self.num_theil_u_weights[col]

        # get predictions
        preds = df[self.cat_selected_features + self.num_selected_features].mean(axis=1)
        df["preds_class_1"] = preds
        df["preds_class_0"] = 1 - preds
        self.df = df
        preds = pd.DataFrame(self.softmax(df[["preds_class_1", "preds_class_0"]].values), columns=["preds_class_1", "preds_class_0"])
        return preds["preds_class_1"]


class ProbaBoost:
    """
    This classifier measures the conditional probability of the target variable for each feature
    separately. Numerical values are converted into buckets for this purpose.
    Additionally, the explanatory power of each feature is measured using Theil's U. The conditional probabilities
    are then weighted by Theil's U.
    :param df: Pandas DataFrame containing the target feature.
    :param cat_feats: List of strings with column names specifying categorical features.
    :param num_feats: List of strings with column names specifying numerical features.
    :param target: String specifying name of the target column.
    """

    def __init__(self,
                 df: pd.DataFrame,
                 cat_feats: list,
                 num_feats: list,
                 target: str,
                 nb_boosters: int = 30,
                 nb_buckets: int = 20,
                 random_state=5000):
        self.df = df
        self.cat_feats = cat_feats
        self.num_feats = num_feats
        self.target = target
        self.nb_boosters = nb_boosters
        self.nb_buckets = nb_buckets
        self.random_state = random_state
        self.conditional_classifiers = []

    def train_model(self, i, tcss):      
        
        sample_df = self.df.sample(tcss, random_state=self.random_state+i, replace=True)
        sample_df = sample_df.reset_index(drop=True)

        cpc = ConditionProbabilitiesClassifier(
            df=sample_df,
            cat_feats=self.cat_feats,
            num_feats=self.num_feats,
            target=self.target,
            nb_buckets=self.nb_buckets)

        cpc.fit()
        return {"model_obj": cpc,
                "nb_buckets": self.nb_buckets}

    def model_predict(self, i, df):
        df = df.reset_index(drop=True)
        cpc = self.conditional_classifiers[i]
        churn_proba = cpc.predict(df)
        del cpc
        _ = gc.collect()
        preds = {}
        preds[i] = churn_proba
        return preds

    def fit(self):
        model_objects = Parallel(n_jobs=-1)(
            delayed(self.train_model)(i, len(self.df.index)) for i in range(self.nb_boosters))

        for obj in model_objects:
            if len(obj["model_obj"].cat_feats + obj["model_obj"].num_feats) > 0:
                self.conditional_classifiers.append(obj["model_obj"])

        del model_objects
        _ = gc.collect()

    def predict(self, df):       
        prediction_objects = Parallel(n_jobs=-1)(
            delayed(self.model_predict)(i, df) for i in range(self.nb_boosters))

        predictions = {}
        for i in prediction_objects:
            for key, item in i.items():
                predictions[key] = item

        del prediction_objects
        _ = gc.collect()

        pred_df = pd.DataFrame(predictions)


        pred_df["predictions"] = pred_df.mean(axis=1)
        pred_df["predictions"] = pred_df["predictions"].astype(np.float64)
        return pred_df["predictions"]

In [13]:
num_cols = train.drop(target, axis=1).columns.to_list()

cbd = ProbaBoost(
        df = train.fillna(0),
        cat_feats=[],
        num_feats=num_cols,
        target=target,
        nb_boosters=30,
        nb_buckets=10
    )
    
cbd.fit()

In [14]:
fraud_cases = cbd.predict(test.fillna(0))

/tmp/ipykernel_18/3764380943.py:292: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
/tmp/ipykernel_18/3764380943.py:292: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
/tmp/ipykernel_18/3764380943.py:292: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
/tmp/ipykernel_18/3764380943.py:292: FutureWarning: Downcasti

# Submission time

In [15]:
submission[target] = fraud_cases
submission.to_csv('submission.csv', index=False)

In [16]:
submission

,id,IsFraud
0,150000,5.962736
1,150001,5.963122
2,150002,5.963556
3,150003,5.963276
4,150004,5.963359
...,...,...
69124,219124,5.970581
69125,219125,5.965511
69126,219126,5.972369
69127,219127,5.965504
